In [1]:
import pandas as pd
from pathlib import Path
from collections import defaultdict

In [2]:
INDEX = "SP500"
folder_path = Path(f"data{INDEX}")
files = [f.name for f in folder_path.iterdir() if f.is_file()]

In [5]:
posts_per_day = defaultdict(lambda: [0, 0])   #dict, inizializza a 0 key non esistenti

for file in folder_path.iterdir():
    ticker = file.stem.split("_")[0]
    try:
        df = pd.read_csv(file)
        if not df.empty:
            if "comments" in file.stem:
                df["date"] = pd.to_datetime(df["commentDate"], errors="coerce")
            else:
                df["date"] = pd.to_datetime(df["videoDate"], errors="coerce")
            time_delta = ( df["date"].max() - df["date"].min() ).days + 1
            
            posts_per_day[ticker][0] += round( len(df) / time_delta , 1 )
            posts_per_day[ticker][1] += len(df)


    except Exception as e:
        print(f"Errore nel file {file.name}: {e}")

In [6]:
data = []
for ticker, counts in posts_per_day.items():
    data.append({ "Ticker": ticker, "TotElems": counts[1], "NumElemsPerDay": round( counts[0], 1), "NumElemsPerWeek": round( counts[0]*7, 1) })


In [8]:
df = pd.DataFrame( data )
df.to_csv( f"num_elem_{INDEX}.csv", index=None )